In [11]:
!pip install -U sentence-transformers
!pip install torch
!pip install json
!pip install transformers

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [12]:
from sentence_transformers import CrossEncoder
import os
import json
import bs4
import re
import numpy as np

In [13]:
# load in the Bi-Encoder results
def load_json_file(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

top_100_results_path = 'formatted_top_100_results.json'

In [14]:
# Cleaning Strings Methods

def clean_topics_text(item):

    full_string = item['Title'] + ' ' + item['Body'] + ' ' + item['Tags']

    full_string = bs4.BeautifulSoup(full_string, 'html.parser').text
    full_string = re.sub(r'[^a-zA-Z\s]', ' ', full_string)

    return full_string

def clean_answers_text(item):
    full_string = bs4.BeautifulSoup(item['Text'], 'html.parser').text
    full_string = re.sub(r'[^a-zA-Z\s]', ' ', full_string)

    return full_string

In [15]:
# get query test from ID

def query_id_to_text(query_id, file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Assuming data is a list of dictionaries
    for item in data:
        if item.get('Id') == query_id:
            text = clean_topics_text(item)
            return text

    return "Query ID not found"

In [17]:
# format the top 100 results

def get_100_queries(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    cleaned_top_100 = {}

    # dict id, query_text -> id, answers_texts, scores
    for query in data:
        query_text = query_id_to_text(query['topic_id'], 'topics_1.json')
        answers_results = []
        for result in query['results']:
            answer = result[0]  # Access the dictionary within the list
            cleaned_text = clean_answers_text(answer)
            answers_results.append({
                'answer_id': answer['Id'],
                'text': cleaned_text
            })
        cleaned_top_100[query['topic_id']] = {
            'query_text': query_text,
            'answers': answers_results
        }

    return cleaned_top_100

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

re_ranked = {}

get_100_queries = get_100_queries(top_100_results_path)


for query_id, query_data in get_100_queries.items():
    query_text = query_data['query_text']
    answers = query_data['answers']
    scores = cross_encoder.predict([(query_text, answer['text']) for answer in answers])
    scored_answers = [{
        'answer_id': answers[i]['answer_id'],
        'text': answers[i]['text'],
        'score': float(scores[i])
    } for i in range(len(answers))]
    re_ranked[query_id] = {
        'query_text': query_text,
        'answers': sorted(scored_answers, key=lambda x: x['score'], reverse=True)
    }

# Save re_ranked dictionary to a JSON file
with open('re_ranked_results.json', 'w') as json_file:
    json.dump(re_ranked, json_file, indent=4)

print("Re-ranked results have been saved to 're_ranked_results.json'")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Re-ranked results have been saved to 're_ranked_results.json'
